# 8. 데이터프레임 수직 결합
- 두 테이블의 구조가 비슷하고, 동일한 열과 자료형이 있는 경우가 거의 대부분 >> 취합용도
- 단, 특정 시점을 전후로 업무 방식이 바뀌기도 하므로 유의
- 데이터의 수직적 결합을 이어붙이기(concatenating) 또는 추가(appending)이라고 함

In [21]:
import pandas as pd
import numpy as np
import os

In [22]:
ltcameroon = pd.read_csv('data/ltcountry/ltcameroon.csv')
ltpoland = pd.read_csv('data/ltcountry/ltpoland.csv')

### 카메룬과 폴란드 데이터 이어붙이기

In [23]:
ltcameroon.shape

(48, 11)

In [24]:
ltpoland.shape

(120, 11)

In [25]:
ltall = pd.concat([ltcameroon, ltpoland])

In [26]:
ltall.country.value_counts()

Poland      120
Cameroon     48
Name: country, dtype: int64

### 국가별 파일을 모두 이어붙이기
- 폴더 내의 국가별 csv파일 전체에 대해 파일명으로 루핑
- 각 파일의 확장자가 csv인지 endswith 메서드로 확인
- 새로운 데이터프레임 행을 concat으로 기존 데이터프레임에 추가
- 마지막으로, 최근의 데이터프레임에 누락값이 있었던 열 또는 기존에 없던 새로운 열이 있으면 출력
- ltoman 데이터프레임에는 latabs열이 없음 유의!

In [27]:
directory = 'data/ltcountry'

In [28]:
ltall = pd.DataFrame()

In [29]:
for filename in os.listdir(directory):  #지정된 디렉토리에서 파일 목록을 가져옴
    if filename.endswith(".csv"):
        fileloc = os.path.join(directory, filename)   #디렉토리 경로와 파일 이름을 연결
    
    # 파일을 열기 위해 with문을 사용
    with open(fileloc) as f:
        ltnew = pd.read_csv(fileloc)
        print(filename + ' has ' + str(ltnew.shape[0]) + ' rows.')
        ltall = pd.concat([ltall, ltnew])
    
    # 열에 차이가 있는지 확인
    # symmetric_difference : 두 집합(또는 열 이름의 집합) 간의 대칭 차집합을 계산. 즉, 한 데이터 프레임에만 있는 열 이름들을 찾음.
    columndiff = ltall.columns.symmetric_difference(ltnew.columns)
    if (not columndiff.empty):
        print("", "Differenct column names for:", filename, columndiff, "", sep="\n")

ltbrazil.csv has 1104 rows.
ltcameroon.csv has 48 rows.
ltindia.csv has 1056 rows.
ltpoland.csv has 120 rows.
ltjapan.csv has 1800 rows.
ltmexico.csv has 852 rows.
ltoman.csv has 288 rows.

Differenct column names for:
ltoman.csv
Index(['latabs'], dtype='object')



### 결합된 데이터 일부를 표시

In [30]:
ltall[['country', 'station', 'month', 'temperature', 'latitude']].sample(5, random_state=1)

,country,station,month,temperature,latitude
717,Brazil,TAGUATINGA,8,27.75,-12.400
529,Japan,HACHIJOJIMA,4,15.30,33.117
52,Oman,BURAIMI_AUT,3,22.40,24.233
1029,India,JAIPUR_SANGANER,12,14.92,26.817
231,Mexico,MANZANILLO,4,24.85,19.050


### 이어붙인 데이터 값 검사
- 오만의 latabs 값들이 모두 누락됨 (위도의 절댓값)

In [31]:
ltall.country.value_counts().sort_index()

Brazil      1104
Cameroon      48
India       1056
Japan       1800
Mexico       852
Oman         288
Poland       120
Name: country, dtype: int64

In [32]:
ltall.groupby(['country']).agg({'temperature':['min', 'mean', 'max', 'count'],
                               'latabs':['min', 'mean', 'max', 'count']})

temperature                          latabs                         
                 min       mean    max count     min       mean     max count
country                                                                      
Brazil         12.20  25.470196  33.93   969   0.051  13.859272  33.517  1104
Cameroon       21.87  27.227941  36.00    34   4.020   7.789250  10.451    48
India           1.95  26.113477  37.40  1044   8.300  21.104516  34.083  1056
Japan          -7.15  15.495854  30.14  1797  24.300  35.664822  45.417  1800
Mexico          7.00  23.230397  34.45   806  14.917  22.114901  32.400   852
Oman           12.10  27.753463  38.39   205     NaN        NaN     NaN     0
Poland         -3.85  10.462750  23.25   120  50.078  52.498500  54.750   120

### 누락값 수정
- 오만의 위도값을 설정

In [33]:
ltall['latabs'] = np.where(ltall.country=='Oman', ltall.latitude, ltall.latabs)

In [34]:
ltall.groupby(['country']).agg({'temperature':['min', 'mean', 'max', 'count'],
                               'latabs':['min', 'mean', 'max', 'count']})

temperature                          latabs                         
                 min       mean    max count     min       mean     max count
country                                                                      
Brazil         12.20  25.470196  33.93   969   0.051  13.859272  33.517  1104
Cameroon       21.87  27.227941  36.00    34   4.020   7.789250  10.451    48
India           1.95  26.113477  37.40  1044   8.300  21.104516  34.083  1056
Japan          -7.15  15.495854  30.14  1797  24.300  35.664822  45.417  1800
Mexico          7.00  23.230397  34.45   806  14.917  22.114901  32.400   852
Oman           12.10  27.753463  38.39   205  16.933  21.997875  26.167   288
Poland         -3.85  10.462750  23.25   120  50.078  52.498500  54.750   120